In [1]:
from models.decision_tree import DecisionTreeModel
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import f1_score, roc_auc_score
from metrics import cross_validate_with_resampling, Metrics
from imblearn.combine import SMOTEENN
from imblearn.over_sampling import RandomOverSampler, SMOTE
import pandas as pd
import numpy as np

In [2]:
%load_ext autoreload
%autoreload 2

In [4]:
decision_tree_model = DecisionTreeModel(name='DecisionTree')
print(decision_tree_model)
decision_tree_model._is_train()

DecisionTree


False

In [3]:
df = pd.read_csv('./data/Telco-Customer-Churn-encoded-data-FE.csv')
label = pd.read_csv('./data/Telco-Customer-Churn-encoded-label.csv')
df.shape, label.shape

((7043, 43), (7043, 1))

In [7]:
estimator = DecisionTreeModel().model
cross_val_score(estimator, df, label, cv = 5, scoring='roc_auc').mean()

0.6588721476177273

In [4]:
X = df.to_numpy()
y = label.to_numpy().ravel()

In [10]:
X

array([[0.00000000e+00, 0.00000000e+00, 1.00000000e+00, ...,
        0.00000000e+00, 1.00000000e+00, 2.98500000e+01],
       [1.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        2.00000000e+00, 3.01402487e-02, 5.55735294e+01],
       [1.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 4.97919556e-01, 5.40750000e+01],
       ...,
       [0.00000000e+00, 0.00000000e+00, 1.00000000e+00, ...,
        0.00000000e+00, 8.54380141e-02, 3.14954545e+01],
       [1.00000000e+00, 1.00000000e+00, 1.00000000e+00, ...,
        0.00000000e+00, 2.42661448e-01, 7.66500000e+01],
       [1.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        5.00000000e+00, 1.54357513e-02, 1.03704545e+02]])

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [10]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((5634, 43), (1409, 43), (5634,), (1409,))

In [15]:
sm = SMOTE(sampling_strategy="all")
Xr_train, yr_train = sm.fit_resample(X_train, y_train)

In [16]:
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)
print("Xr_train shape:", Xr_train.shape)
print("yr_train shape:", yr_train.shape)

X_train shape: (5634, 43)
X_test shape: (1409, 43)
y_train shape: (5634, 1)
y_test shape: (1409, 1)
Xr_train shape: (8276, 43)
yr_train shape: (8276,)


In [17]:
decision_tree_modelr = DecisionTreeModel(name='DecisionTree')

In [18]:
cross_validate_with_resampling(DecisionTreeModel(), X, y)

(0.6501798702605038, 0.6566606326729503)

In [19]:
decision_tree_model.fit(X_train, y_train)
decision_tree_modelr.fit(Xr_train, yr_train)
y_pred = decision_tree_model.predict(X_test)
yr_pred = decision_tree_modelr.predict(X_test)


roc_auc_score(yr_pred, y_test)

0.6452306060961374

In [20]:
roc_auc_score(y_pred, y_test)


0.6658975938285691